In [15]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import time

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.metrics import mean_squared_error

from math import sqrt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
df = pd.read_csv("dataset/train_processed.csv")

In [3]:
df.shape

(501300, 49)

We normalize the data so that all values are between 0 and 1. So we first remove 'Date' which is the only non numerical attribute. Then we apply range normalization and at the end we add 'Date'. 

In [4]:
df_date=df['Date']
df_no_date=df.drop(['Date'], axis=1)
df_norm= (df_no_date - df_no_date.min()) / (df_no_date.max() - df_no_date.min())
df=df_norm
df['Date']=df_date
df.head(5)

,Unnamed: 0,StoreID,IsHoliday,IsOpen,HasPromotions,StoreType,NearestCompetitor,NumberOfCustomers,NumberOfSales,Region_AreaKM2,...,Thunderstorm,IsSaturday,IsSunday,WasOpenYesterday,IsOpenTomorrow,YesterdaySales,Previous3DaysSales,PreviousWeekSales,PreviousMonthSales,Date
0,0.000000,0.0,0.0,1.0,1.0,0.666667,0.003281,0.383500,0.446943,0.291715,...,0.0,0.0,0.0,1.0,0.0,0.339890,0.363301,0.355788,0.372606,2016-03-31
1,0.000002,0.0,1.0,0.0,1.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,1.0,1.0,0.446943,0.409861,0.384367,0.384941,2016-04-01
2,0.000004,0.0,0.0,1.0,0.0,0.666667,0.003281,0.385766,0.417740,0.291715,...,0.0,1.0,0.0,0.0,0.0,0.000000,0.290735,0.326042,0.368885,2016-04-02
3,0.000006,0.0,0.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,1.0,1.0,0.0,0.417740,0.319501,0.363920,0.377885,2016-04-03
4,0.000008,0.0,1.0,0.0,0.0,0.666667,0.003281,0.000000,0.000000,0.291715,...,0.0,0.0,0.0,0.0,1.0,0.000000,0.154355,0.363920,0.362002,2016-04-04


We split the dataset in 4 parts:
* df_train: the training set (all the rows before 2017-09-01). We use it to train our model.
* df_val1: the first validation set (all the rows between 2017-09-01 and 2017-11-01). We use it to tune the hyperparameter of our model.
* df_val2: the second validation set (all the rows between 2017-11-01 and 2018-01-01). We use it to select the best model.
* df_val3: the third validation set (all the rows after 2018-01-01). We use it to check that our final model works properly.

In [5]:
df_val3=df.loc[(df['Date']>='2018-01-01')]
df_val3_rest=df.loc[(df['Date']<'2018-01-01')]
df_val2=df_val3_rest.loc[(df_val3_rest['Date']>='2017-11-01')]
df_val2_rest=df.loc[(df['Date']<'2017-11-01')]
df_val1=df_val2_rest.loc[(df_val2_rest['Date']>='2017-09-01')]
df_train=df.loc[(df['Date']<'2017-09-01')]

We remove 'Date' from our set because it is only used as index

In [6]:
df_train_clear=df_train.drop(['Date'], axis=1)
df_val1_clear=df_val1.drop(['Date'], axis=1)
df_val2_clear=df_val2.drop(['Date'], axis=1)
df_val3_clear=df_val3.drop(['Date'], axis=1)

We try to find the best number of features

In [41]:
y=df_train_clear['NumberOfSales']
y_val1=df_val1_clear['NumberOfSales']
y_val2=df_val2_clear['NumberOfSales']
y_val3=df_val3_clear['NumberOfSales']

In [49]:
X_t = df_train_clear.loc[df_train_clear['StoreID'] == 0.0]
y_t = X_t['NumberOfSales']
y_val_t = df_val1_clear.loc[df_val1_clear['StoreID'] == 0.0]['NumberOfSales']
X_t = X_t.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)

In [51]:
y_val_t.shape

(61,)

In [22]:
X=df_train_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val1=df_val1_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val2=df_val2_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)
X_val3=df_val3_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers','YesterdaySales','Previous3DaysSales','PreviousWeekSales','PreviousMonthSales'], axis=1)

In [14]:
dict_df = {'loss_function' : [], 'alpha' : [], 'epsilon' : [], 'learning_rate' : [], 'var_threshold' : [],'num_feat' : [], 'mse' : []}

In [36]:
sfm = SelectFromModel(sgd, threshold=0.03)
sfm.fit(X, y)
sfm.transform(X).shape[1]

41

In [ ]:
loss_fun = ['squared_loss', 'epsilon_insensitive']
learning_r = ['constant', 'optimal', 'invascaling']

start = time.time()

for l_f in loss_strategy:
    for l_r in learning_rate:
        for a in np.arange(.0001, .01 , .0001):
            for e in np.arange(.1, .5, .1):
                sgd = SGDRegressor(loss_function=l_f, alpha=a, epsilon=e, learning_rate=l_r, shuffle=False, random_state=1234567890)
                dict_df['loss_function'].append(l_f)
                dict_df['alpha'].append(a)
                dict_df['epsilon'].append(e)
                dict_df['learning_rate'].append(l_r)
                for v in np.arange(.001, .03, .0005):
                    sfm = SelectFromModel(sgd, threshold=v)
                    sfm.fit(X, y)
                    n_features = sfm.transform(X).shape[1]
                    dict_df['var_threshold'].append(v)
                    dict_df['num_feat'].append(n_features)
                    if n_features == 0:
                        dict_df['mse'].append(99999999999999999)
                    else:
                        mask = sfm.get_support()
                        X_mask = X.loc[:, mask]
                        X_val1_mask = X_val1.loc[:, mask]
                        lasso.fit(X_mask, y)
                        y_pred=lasso.predict(X_val1_mask)
                        dict_df['mse'].append(sqrt(mean_squared_error(y_val1,y_pred)))
            
end = time.time() - start
end